In [11]:
!pip install pandas
!pip install scikit-learn
!pip install textblob
!pip install colorama

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [13]:
import pandas as pd
import random
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
from colorama import init, Fore

init(autoreset=True)

# -------------------- Load & Prepare Data --------------------
def load_data(file_path="imdb.csv"):
    df = pd.read_csv(file_path)
    df['Genre'] = df['Genre'].fillna('')
    df['Overview'] = df['Overview'].fillna('')
    df['combined_features'] = df['Genre'] + " " + df['Overview']
    return df

movies_df = load_data()

# -------------------- TF-IDF & Similarity --------------------
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['combined_features'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# -------------------- Utilities --------------------
def list_genres(df):
    return sorted(
        set(g.strip() for sub in df['Genre'].str.split(', ') for g in sub if g)
    )

genres = list_genres(movies_df)

def processing_animation():
    for _ in range(3):
        print(Fore.YELLOW + ".", end="", flush=True)
        time.sleep(0.4)
    print()

# -------------------- AI Recommendation --------------------
def recommend_movies(genre=None, mood=None, rating=None, top_n=5):
    df = movies_df.copy()

    if genre:
        df = df[df['Genre'].str.contains(genre, case=False, na=False)]

    if rating:
        df = df[df['IMDB_Rating'] >= rating]

    df = df.sample(frac=1).reset_index(drop=True)
    recommendations = []

    for _, row in df.iterrows():
        polarity = TextBlob(row['Overview']).sentiment.polarity

        if mood:
            mood_polarity = TextBlob(mood).sentiment.polarity
            if mood_polarity < 0 and polarity <= 0:
                continue

        recommendations.append((
            row['Series_Title'],
            row['Genre'],
            row['IMDB_Rating'],
            polarity
        ))

        if len(recommendations) == top_n:
            break

    return recommendations if recommendations else None

# -------------------- Random Recommendation --------------------
def random_movie():
    row = movies_df.sample(1).iloc[0]
    polarity = TextBlob(row['Overview']).sentiment.polarity
    return row['Series_Title'], row['Genre'], row['IMDB_Rating'], polarity

# -------------------- Display --------------------
def display_movie(movie):
    title, genre, rating, polarity = movie
    sentiment = "Positive 😊" if polarity > 0 else "Negative 😔" if polarity < 0 else "Neutral 😐"

    print(Fore.CYAN + f"\n🎬 Title: {title}")
    print(Fore.GREEN + f"🎭 Genre: {genre}")
    print(Fore.YELLOW + f"⭐ IMDB Rating: {rating}")
    print(Fore.MAGENTA + f"💬 Sentiment: {sentiment} ({polarity:.2f})")

# -------------------- Main Interaction --------------------
def main():
    print(Fore.BLUE + "🎥 AI Movie Recommendation System\n")
    name = input(Fore.YELLOW + "Enter your name: ").strip()
    print(Fore.GREEN + f"\nWelcome, {name}!\n")

    while True:
        print(Fore.CYAN + "Choose an option:")
        print("1. AI-based Recommendation")
        print("2. Random Recommendation")
        choice = input("Enter choice (1/2): ").strip()

        if choice == "1":
            print(Fore.GREEN + "\nAvailable Genres:")
            for i, g in enumerate(genres, 1):
                print(f"{i}. {g}")

            genre_input = input("\nEnter genre name (or press Enter to skip): ").strip()
            mood = input("How do you feel today? ").strip()

            rating_input = input("Minimum IMDB rating (or skip): ").strip()
            rating = float(rating_input) if rating_input.replace('.', '', 1).isdigit() else None

            print(Fore.BLUE + "\nFinding movies", end="")
            processing_animation()

            recs = recommend_movies(genre_input, mood, rating)
            if not recs:
                print(Fore.RED + "No suitable movies found.")
            else:
                for movie in recs:
                    display_movie(movie)

        elif choice == "2":
            print(Fore.BLUE + "\nPicking a random movie", end="")
            processing_animation()
            display_movie(random_movie())

        else:
            print(Fore.RED + "Invalid choice.")
            continue

        again = input(Fore.YELLOW + "\nWant another recommendation? (yes/no): ").lower()
        if again != "yes":
            print(Fore.GREEN + f"\nEnjoy your movie time, {name}! 🍿")
            break

# -------------------- Run --------------------
if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'textblob'